#PROPOSITO DE ESTE CUADERNO:
Ser utilizado para el desarrollo del proyecto final del [__certificado profesional "Ciencia de datos de IBM"__](https://www.coursera.org/professional-certificates/certificado-profesional-de-ciencia-de-datos-de-ibm) disponible en la plataforma [__Coursera.__](https://www.coursera.org)

##Creando el entregable de la semana 1 - (*5% de la nota final*)
Corresponde al tema: ***Notebook del Proyecto Final***



In [1]:
import pandas as pd
import numpy as np

In [2]:
print('¡Hola, Proyecto Final del Curso!')

¡Hola, Proyecto Final del Curso!


* * *
##Creando el entregable de la semana 2 - (*10% de la nota final*)
Corresponde a un examen sobre Foursquare API

***
##Creando el entregable de la semana 3 - (*15% de la nota final*)
correspondiente al tema: ***Agrupación y segmentación de vecindarios en la ciudad de Toronto, Canadá***

Lo primero que toca es agregar las librerias que hace falta para poder realizar la tarea de extraer datos de una url

In [3]:
from bs4 import BeautifulSoup
import requests

Luego, debemos obtener el contenido HTML de la página web y procesarlo para obtener la tabla que contiene la información de los códigos postales de Canadá.

In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find('table')

Una vez que tenemos la tabla, podemos iterar sobre sus filas y columnas para extraer la información de cada celda.

In [5]:
data = []

# Obtenemos todas las filas omitiendo aquellas que poseen el texto 'Not assigned' y separamos con una ',' los Neighborhoods que pertenecen a un mismo Borough
for row in table.find_all('td'):
    cell = {}
    if row.span.text == 'Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        data.append(cell)

Finalmente, podemos crear un DataFrame a partir de la lista de datos e imprimir su numero de filas.

In [6]:
# Creamos un DataFrame a partir de la lista de datos
df = pd.DataFrame(data)

# Imprimimos el número de filas del DataFrame
print('Número de filas:', df.shape[0])

Número de filas: 103


Dataframe resultante:

In [8]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [10]:
# Guardando dataframe actual en un archivo .csv -- Hasta este punto esta cubierto el primer entregable con ponderacion maxima de 10 puntos
df.to_csv('df_week3_10-points.csv', index=False)

Para agregar las columnas Latitud y Longitud al dataframe existente usando la columna PostalCode como clave de unión, haremos lo siguiente:

1. Cargar el archivo Geospatial_Coordinates.csv en un nuevo dataframe, usando la función pandas.read_csv(). 
IMPORTANTE!!! Debemos asegurarnos que la columna PostalCode tenga el mismo nombre en ambos dataframes.

In [11]:
# Carga el archivo CSV en un nuevo dataframe
df_lat_long = pd.read_csv('Geospatial_Coordinates.csv')

2. Usar la función merge() de Pandas para unir los dos dataframes en función de la columna PostalCode.

In [12]:
# Fusiona los dos dataframes en base a la columna PostalCode
df_merged = pd.merge(df, df_lat_long, on='PostalCode', how='left')

La función merge() toma dos dataframes y los une en función de una columna común, en este caso, PostalCode. La opción how='left' indica que se conservarán todas las filas del dataframe original y las filas que no tienen una coincidencia en el dataframe de latitud y longitud se rellenarán con valores nulos.

3. Verificar que se hayan agregado correctamente las columnas Latitud y Longitud al dataframe fusionado.

In [13]:
# Verificando que se hayan agregado las columnas Latitud y Longitud
print(df_merged.head())

  PostalCode           Borough                      Neighborhood   Latitude  \
0        M3A        North York                         Parkwoods  43.753259   
1        M4A        North York                  Victoria Village  43.725882   
2        M5A  Downtown Toronto         Regent Park, Harbourfront  43.654260   
3        M6A        North York  Lawrence Manor, Lawrence Heights  43.718518   
4        M7A      Queen's Park     Ontario Provincial Government  43.662301   

   Longitude  
0 -79.329656  
1 -79.315572  
2 -79.360636  
3 -79.464763  
4 -79.389494  


In [14]:
# Guardando df fusionado --- Hasta este punto esta cubierto el segundo entregable con ponderacion maxima de 2 puntos
df_merged.to_csv('df_merged_for_week-3.csv', index=False)

En Python, hay varias librerías que podemos usar para explorar y visualizar datos. Para este caso, utilizaré pandas y folium.

Primero, vamos a importar la librería folium y usaremos el dataframe fusionado elaborado en el paso anterior

In [15]:
import folium

La columna 'Borough' contiene los nombres de los municipios, podemos crear una nueva columna que contenga un valor booleano indicando si el valor contiene 'Toronto' o no:

In [16]:
df_merged['Toronto'] = df_merged['Borough'].str.contains('Toronto')


Luego, podemos filtrar el dataframe para obtener sólo las filas que contengan 'Toronto':

In [17]:
toronto_df = df_merged[df_merged['Toronto'] == True]


Ahora, podemos agrupar por municipio (Borough):

In [19]:
grouped_df = toronto_df.groupby(['Borough']).agg({'PostalCode': 'count', 'Latitude': 'mean', 'Longitude': 'mean'})
grouped_df.rename(columns={'PostalCode': 'Cantidad'}, inplace=True)

Esto nos dará un nuevo dataframe llamado grouped_df que contiene la cantidad de códigos postales por barrio, así como la latitud y longitud promedio para cada barrio.

Finalmente, podemos crear un mapa utilizando folium:

In [20]:
mapa = folium.Map(location=[43.6532, -79.3832], zoom_start=12)

for lat, lng, borough in zip(grouped_df['Latitude'], grouped_df['Longitude'], grouped_df.index):
    label = folium.Popup(borough, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(mapa)

mapa

Esto creó un mapa centrado en Toronto con marcadores para cada municipio, donde el tamaño y el color de cada marcador indica la cantidad de códigos postales en ese municipio.

Con esto concluye la tercera parte de este entregable. La parte del mapa tiene una ponderacion maxima de 3 puntos.